<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. Transformer

## 7.2 Finetuning BERT

#### Setup

Zuerst muessen wir die ueblichen Vorbereitungen treffen

In [ ]:
# install transformers library
!pip install transformers

     |████████████████████████████████| 481kB 5.0MB/s 
     |████████████████████████████████| 870kB 24.1MB/s 
     |████████████████████████████████| 1.0MB 35.9MB/s 
     |████████████████████████████████| 3.1MB 47.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=293149735527043eaeb4a9a8a1e8e5cbe6793dd636b8a855f8959dfeb42ec6e3
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [ ]:
# necessary imports

%tensorflow_version 2

import tensorflow_datasets as tfds
import torch.utils.data as datatools
import transformers
import torch
import os

from pprint import pprint

torch.cuda.get_device_name()

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


'Tesla P100-PCIE-16GB'

Als naechste mounten wir Google Drive und legen uns einen Ordner in dem wir Daten/Modelle zwischenspeichern

In [ ]:
# connect google drive

from google.colab import drive
drive.mount('/content/drive')

# Hier werden wir Daten speicher und laden
model_dir = '/content/drive/My Drive/Squad/' 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Laden des Modell

Nu legen wir fest, welches Modell wir verwenden wollen. Dabei koennen wir entweder auf ein vortrainiertes Modell aus der Transfomers-Bibiliothek zugreifen, oder ein von uns zuvor trainiertes von der Platte laden.

In [ ]:
# Namen eines Modell der TransfomerBibliothek oder Pfad zum Modell auf der Platte
model_name_or_path = 'bert-base-uncased' 
# wenn wir eines unserer Modelle weitertrainieren wollen können wir hier auch den Pfad zu diesem
# Modell angeben
# model_name_or_path = '/content/drive/My Drive/Squad/0'

# Jetzt leiten wir aus den Modell Pfad die letzte Trainingsepoche ab
# oder setzen diese auf -1 wenn wir mit dem Modelltraining von vorne beginnen
last_epoch = -1 if 'drive' not in model_name_or_path else int(model_name_or_path.split('/')[-1])

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name_or_path)
_ = model.to('cuda')

#### Erstellen des Datensatzes 

In [ ]:
#@title 
#@markdown Versteckter Code: Herunterladen und vorbereiten des Squad Datensatzes

# Here we define the dataset preprocessing
dataset_parameter = dict(
    doc_stride = 128, # When splitting up a long document into chunks, how much stride to take between chunks.
    max_seq_length = 384, # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated
    max_query_length= 64, # The maximum number of tokens for the question. Questions longer than this will be truncated to this length."
    return_dataset="pt", # we generate data in the pytorch format
)

cache_file = os.path.join(model_dir, 'train_cache')
if not os.path.exists(cache_file):

    # dowload dataset
    tfds_examples = tfds.load("squad")
    # prepare samples
    train_examples = transformers.SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=False)
    train_features, train_dataset = transformers.squad_convert_examples_to_features(
        examples=train_examples,
        tokenizer=tokenizer,
        is_training=True,
        **dataset_parameter
    )
    torch.save({"features": train_features, "dataset": train_dataset, "examples": train_examples}, cache_file)
else: # load data
    features_and_dataset = torch.load(cache_file)
    train_features, train_dataset, train_examples = (
        features_and_dataset["features"],
        features_and_dataset["dataset"],
        features_and_dataset["examples"],
    )

INFO:absl:No config specified, defaulting to first: squad/plain_text
INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for squad/plain_text/1.0.0
INFO:absl:Generating dataset squad (/root/tensorflow_datasets/squad/plain_text/1.0.0)


INFO:absl:Downloading https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json into /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1uLsZc14btZFRCgHMAy9Mn5abwO6wga4bMozTBvOyQAg.json.tmp.b71cc92e294d416d86669b09f9bd4cb3...
INFO:absl:Downloading https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json into /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1nVRimH71-BT-FaNpwXJPbsOaIBizticanX0lmGhsov8.json.tmp.36d537ca404f4df2b32e75bcdfee1f78...
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urll

INFO:absl:generating examples from = /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1NSdmOYa4KVr09_zf8bof8_ctB9YaIPSHyyOKbvkv2VU.json


Shuffling and writing examples to /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete2ZWOYM/squad-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete2ZWOYM/squad-train.tfrecord. Shard lengths: [87599]
INFO:absl:Generating split validation


INFO:absl:generating examples from = /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1lapqUtXWpzVWM2Z1PKUEkqZYAx2nTzAaxSOLA5Zpcsk.json


Shuffling and writing examples to /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete2ZWOYM/squad-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete2ZWOYM/squad-validation.tfrecord. Shard lengths: [10570]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/squad/plain_text/1.0.0


Dataset squad downloaded and prepared to /root/tensorflow_datasets/squad/plain_text/1.0.0. Subsequent calls will reuse this data.


87599it [01:17, 1132.16it/s]
add example index and unique id: 100%|██████████| 87599/87599 [00:00<00:00, 685282.28it/s]


In [ ]:
example_id = 0
example = train_features[example_id]

pprint(''' ============= Input =============''')
pprint(tokenizer.decode(example.input_ids))

print()
pprint(''' ============= Desired Output =============''')
print(f'Anwort von Token {example.start_position} bis Token {example.end_position}')
pprint(tokenizer.decode(example.input_ids[example.start_position:example.end_position+1]))

' ============= Input ============='
('[CLS] what is one use that would require an antenna to receive signals in '
 'various ways at once? [SEP] the difference in the above factors for the case '
 'of θ = 0 is the reason that most broadcasting ( transmissions intended for '
 'the public ) uses vertical polarization. for receivers near the ground, '
 'horizontally polarized transmissions suffer cancellation. for best reception '
 'the receiving antennas for these signals are likewise vertically polarized. '
 'in some applications where the receiving antenna must work in any position, '
 'as in mobile phones, the base station antennas use mixed polarization, such '
 'as linear polarization at an angle ( with both vertical and horizontal '
 'components ) or circular polarization. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] '
 '[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] '
 '[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] '
 '[PAD] [PAD] [PAD] [P

#### Spezifizieren des Trainings (der Optimierung)

(1) Optimierer

Wir benutzen AdamW, einen der besten Optimierer derzeit (https://www.fast.ai/2018/07/02/adam-weight-decay/)

In [ ]:
optimizer = transformers.AdamW(model.parameters(), lr=3e-5, eps=1e-8)

(2) Datenfluss

In [ ]:
train_dataloader = datatools.DataLoader(
    train_dataset, 
    shuffle=True, # Zufällige Aneinandereihung der Trainingsbeispiele
    batch_size=12 # Anahl der Beispiele die gleichzeitig trainiert werden
)

#### (3) Trainingsloop

In [ ]:
# Anzah der Epochen, die wir unser Modell trainieren wollen
NUM_TRAIN_EPOCHS = 3

# Parameter um das Training stabiler zu machern (s.u.)
GRADIENT_ACCUMULATION_STEPS = 4
MAX_GRAD_NORM = 1.0 

for epoch in range(last_epoch+1, last_epoch+1+NUM_TRAIN_EPOCHS):
    print(f"== EPOCH {epoch} started ==" )
    
    train_loss = 0
    model.train() # Am Anfang versetzen wir unser Modell in den Trainingsmodus

    for step, batch in enumerate(train_dataloader):
        
        # (1) Forward-Pass um den Fehler zu berechnen
        batch = tuple(t.to('cuda') for t in batch) # Hier schieben wir unsere Daten auf die GPU 
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
        outputs = model(**inputs)
        loss = outputs[0]
        
        # Buchhaltung des Trainingsfehler
        train_loss += loss.item()

        # Normalisieren des Fehlers
        loss = loss / GRADIENT_ACCUMULATION_STEPS
        
        # (2) Backward-Pass um zu berechnen wie die Parametern verändert werden
        # müssten um den Fehler zu verringern
        loss.backward()
        
        # (3) Anwenden des Optimierers um die Parameter zu verbessern.
        # Wir führen diese Operation nicht nach jedem Forward-Pass durch, sondern
        # nur alle GRADIENT_ACCUMULATION_STEPS um mehr Informationen zu sammeln
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            # Zuerst beschränken wir die Stärke unserer Änderungen um zu starke
            # Ausschläge zu verhindern
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            # Dann führen wir die Optimierung durch
            optimizer.step()
            model.zero_grad()

        # Fehlerausgabe alle 500 Schritte
        if (step+1) % 500 == 0:
            print(f'Train Loss {train_loss/(500):.3f}')
            train_loss = 0

    last_epoch = epoch
    # Speichern des (Zwischen)-Modell
    save_dir = os.path.join(model_dir, str(epoch))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)  

== EPOCH 0 started ==
Train Loss 3.102
Train Loss 1.647
Train Loss 1.494
Train Loss 1.354
Train Loss 1.285
Train Loss 1.291
Train Loss 1.204
Train Loss 1.224
Train Loss 1.158
Train Loss 1.154
Train Loss 1.137
Train Loss 1.121
Train Loss 1.107
Train Loss 1.125
== EPOCH 1 started ==
Train Loss 0.829
Train Loss 0.838
Train Loss 0.859
Train Loss 0.808
Train Loss 0.822
Train Loss 0.841
Train Loss 0.825
Train Loss 0.856
Train Loss 0.823
Train Loss 0.823
Train Loss 0.834
Train Loss 0.833
Train Loss 0.836
Train Loss 0.792
== EPOCH 2 started ==
Train Loss 0.575
Train Loss 0.595
Train Loss 0.576
Train Loss 0.601
Train Loss 0.571
Train Loss 0.587
Train Loss 0.611
Train Loss 0.625
Train Loss 0.571
Train Loss 0.588
Train Loss 0.615
Train Loss 0.589
Train Loss 0.627
Train Loss 0.615


### Aufgabe:
Lasse das Modell für 1 oder 2 Epochen laufen (ca. 3h). Dabei werden in deinem Google-Drive die Ordner `Squad/0` bzw. `Squad/1` angelegt. Trage den Pfad zu diesem Model in dem Squad Evaluationsnotebook unter `model_name_or_path = ` ein und führe dieses komplett aus.

Wie gut funktioniert unser selbst finegetuntes Model?
